# Class 21: Intro to Machine Learning

Plan for today:
- Creating confidence intervals
- Introduction to Machine Learning


In [ ]:
import YData

# YData.download.download_class_code(21)   # get class code    
# YData.download.download_class_code(21, TRUE) # get the code with the answers 

# YData.download.download_homework(9)  # downloads the homework 


If you are using colabs, you should uncomment and run the code below.

In [ ]:
# !pip install https://github.com/lederman/YData_package/tarball/master
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import statistics
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
from urllib.request import urlopen

import matplotlib.pyplot as plt
%matplotlib inline

## 1. Intro to Machine Learning:  Features (X) and labels (y)

In supervised machine learning, we use a computer algorithm called a "pattern classifier" to learn relationships between a set of features X, and a label y. When the classifier is given new examples X, it can then make new predictions y. 


In [ ]:
penguins = sns.load_dataset("penguins")

penguins = penguins.dropna()

penguins = penguins.sample(frac = 1)

penguins.head()

In [ ]:
# Let's explore how many different members there are of each species in our data set? 

species_counts = ...

species_counts


#### Questions: 

1. If we had to guess the species of the penguin without knowing any of the penguin's features, species of penguin should we guess? 
A: Always guess Adelie


2. If we were to following the optimal guessing strategy, what percent of our guess would be correct (i.e., what would our classification accuracy be)? 


In [ ]:
...

To begin the classification process, let's store the features (X) and the labels (y) in separate names called `X_penguin_features` and `y_penguin_labels` respectively. 

In [ ]:
# get the features and the labels

X_penguin_features = penguins[['bill_length_mm', 
                               'bill_depth_mm',
                               'flipper_length_mm', 
                               'body_mass_g']]

y_penguin_labels = penguins['species']


## 2. k-Nearest Neighbors classifier


To explore classification, let's use a k-Nearest Neighbors classifier to predict the species of a penguin based on particular features the penguin has such as the penguin's bill length and body mass. 

Let's construct a K-Nearest Neighbor classifier (KNN) using 5 neighbors for predictions (i.e., k = 5 so we are using a 5-Nearest Neighbor classifier). 

We can do this using the `KNeighborsClassifier(n_neighbors = )` function.  


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Construct a classifier a 5 nearest neighbor classifier
knn = ...


Let's now train the classifier (the KNN classifier just stores the data during training)


In [ ]:
# “train” the classifier (which for a KNN classifier just involves memorizing the training data)

...

Let's now use the classifier to make predictions

In [ ]:
# make predictions
penguin_preditions = ...

Let's get the prediction (classificaton accuracy) which is the proportion of predictions that are correct

In [ ]:
# get the classification accuracy
...

Let's repeat our analysis with k = 1 to see what happens...

In [ ]:
# What happens if k = 1?

# construct a classifier
knn = KNeighborsClassifier(n_neighbors = 1) 

# “train” the classifier (which for a KNN classifier just involves memorizing the training data)
...

# make predictions
...

# get classification accuracy
...

Do we believe we have a perfect classifier???


## 3. Cross-validation

To avoid over-fitting, we need to split our data into a training and test set. 

The classifier "learns" the relationship between features (X) and labels (y) on the **training set**.

The classifier makes predictions on the features (X) of the **test set**. 

We compare the classifier's predictions on the test features (X) to the actual labels y, to get a more accuracy assessment of the **classification accuracy**.


Let's try this now...



In [ ]:
# manually create a training with 250 examples, and a test set that has the rest of the data

...

print(X_train_manual.shape)
print(X_test_manual.shape)


In [ ]:
from sklearn.model_selection import train_test_split

# split data into a training and test set

X_train, X_test, y_train, y_test = ....


print(X_train.shape)
print(X_test.shape)

X_train.head(3)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier


# construct a classifier
knn = ...

# “train” the classifier (which for a KNN classifier just involves memorizing the training data)
...



In [ ]:
# get the predictions

penguin_preditions = ...

penguin_preditions

In [ ]:
# Get the prediction accuracy 

...


In [ ]:
# Test the classifier on the test set using the .score() method

...

In [ ]:
# What happens if we test the classifier on the training set? 

...


### K-fold cross-validation

In k-fold cross-validation we split our data into k-parts (note, the k here has no relation to the k in k-Nearest Neighbor - it is just that k is a frequent letter to use in math to denote integer values).  

To run a k-fold cross-validation analysis, we train the classifier on k-1 parts of the data and test it on the remaining part. We repeat this process k times to get k classification accuracies. We then take the average of these results as our estimate of our overall classification accuracy. 

We can use the scikit-learn `cross_val_score()` to easily do this...


In [ ]:
from sklearn.model_selection import cross_val_score

knn = ...

# do 5-fold cross-validation
scores = ...

print(scores)

print(scores.mean())

## 4. Other classifiers

Many other types of classifiers that have been created. Scikit-learn makes it very easy to try out a range of classifiers. 

Let's explore the Support Vector Machine, and Random Forest Classifier on our penguin data...

In [ ]:
# Suppress ConvergenceWarning if we need it.
#import warnings
#from sklearn.exceptions import ConvergenceWarning
#warnings.filterwarnings("ignore", category=ConvergenceWarning)
#warnings.filterwarnings("ignore", category=FutureWarning)


# Try a support vector machine (SVM)

from sklearn.svm import LinearSVC

svm = ...

scores = cross_val_score(svm, X_penguin_features,  y_penguin_labels, cv = 5)

print(scores.mean())

In [ ]:
# Try a random forest

from sklearn.ensemble import RandomForestClassifier

random_forest = ...

scores = cross_val_score(random_forest, X_penguin_features,  y_penguin_labels, cv = 5)

print(scores.mean())

## 5. Building the KNN classifier

So far we have used the KNN classifier (and a few other classifiers). Let's now see if we can write code that will implement the KNN classifier.

We will do this by writing a several helper functions that build on each other. These functions are: 

1. `euclid_dist(x1, x2)`: finds the Euclidean distance between two points `x1` and `x2`

2. `get_labels_and_distances(test_point, X_train_features, y_train_labels)`: This function finds the distance between a test point and all the training points. It returns a DataFrame with the distance from all training points and the training labels for each point.

3. `classify_point(test_point, k, X_train_features, y_train_labels)`: Classifies which class a test point belongs to

4. `classify_all_test_data(X_test_data, k, X_train_features, y_train_labels)`: Classifiers which class all test points below to.


Let's start by writing a function that can get the Euclidean distance between two points `x` and `z`: 

$$dist(x, z) = \sqrt{\Sigma_{i = 1}^d (x_i - z_i)^2)}$$


In [ ]:
def euclid_dist(x1, x2):
    return ...


# test our function 
my_vec1 = np.array([1, 2, 3, 4])
my_vec2 = np.array([2, 3, 4, 5])

euclid_dist(my_vec1, my_vec2)

In [ ]:
# Let's now write a function that returns the labels and distances 
# between a training point and all the test points


def get_labels_and_distances(test_point, X_train_features, y_train_labels):
    ...



# test our code 

test_data_point = X_test.iloc[0]
test_label = y_test.iloc[0]

labels_and_distances = get_labels_and_distances(test_data_point, X_train, y_train)

labels_and_distances.head(5)

In [ ]:
# get the k closest neighbors

k = 5

sorted_labels_dist = ...

In [ ]:
# get the majority label

...

In [ ]:
# write a function to do the classification on a test point 
# by putting together all the pieces

def classify_point(test_point, k, X_train_features, y_train_labels):
    ...


# test our classifier on one test point

prediction = classify_point(test_data_point, 5, X_train, y_train)

print(prediction)

print(test_label)

In [ ]:
# classify a full test set

def classify_all_test_data(X_test_data, k, X_train_features, y_train_labels):
    ...
    


# test the classifier on the whole test set    

all_predictions = classify_all_test_data(X_test, 5, X_train, y_train)

all_predictions


In [ ]:
# get the classification accuracy

np.mean(all_predictions == y_test)

